In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import cv2
import numpy as np
!pip install -U keras
!pip install tensorflow-gpu==2.1.0rc0

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [0]:
import math
import random
import keras
from keras.layers import *
from keras.models import Sequential
from keras import Model
from keras import backend as K  
from keras.regularizers import l2
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.model_selection import LeaveOneOut
from sklearn import metrics

Using TensorFlow backend.


In [0]:
def get_conv_block(input_layer,nFilters,size):
    conv1 = Conv2D(nFilters, size, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(input_layer)
    bn1 = BatchNormalization()(conv1)
    conv2 = Conv2D(nFilters, size, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(bn1)
    bn2 = BatchNormalization()(conv2)
    return bn2
    
def get_UNET(input_layer,nFilters,flag): 

    block1 = get_conv_block(input_layer[0],nFilters,3)
    mp1 = MaxPooling2D(pool_size=(2, 2))(block1)
    dr1 = Dropout(0.1)(mp1)
   
    if(flag==1):
      dr1 = concatenate([dr1,input_layer[1]])

    block2 = get_conv_block(dr1,nFilters*2,3)
    mp2 = MaxPooling2D(pool_size=(2, 2))(block2)
    dr2 = Dropout(0.1)(mp2)

    if(flag==1):
      dr2 = concatenate([dr2,input_layer[2]])
   
    block3 = get_conv_block(dr2,nFilters*4,3)
    mp3 = MaxPooling2D(pool_size=(2, 2))(block3)
    dr3 = Dropout(0.1)(mp3)

    if(flag==1):
      dr3 = concatenate([dr3,input_layer[3]])
       
    block4 = get_conv_block(dr3,nFilters*8,3)
    mp4 = MaxPooling2D(pool_size=(2, 2))(block4)
    dr4 = Dropout(0.1)(mp4)

    conv5 = Conv2D(nFilters*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(dr4)
    conv5 = Conv2D(nFilters*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(conv5)

    up1 = Conv2DTranspose(nFilters*8,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(conv5)
    cat1 = concatenate([block4, up1, mp3])
    dr1 = Dropout(0.1)(cat1)
    block5 = get_conv_block(dr1,nFilters*8,3)

    up2 = Conv2DTranspose(nFilters*4,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block5)
    b4_upsample = Conv2DTranspose(nFilters*4,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block4)
    cat2 = concatenate([block3, up2, b4_upsample, mp2])
    dr2 = Dropout(0.1)(cat2)
    block6 = get_conv_block(dr2,nFilters*4,3)
    
    up3 = Conv2DTranspose(nFilters*2,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block6)
    b3_upsample = Conv2DTranspose(nFilters*2,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block3)
    cat3 = concatenate([block2, up3, mp1, b3_upsample])
    dr3 = Dropout(0.1)(cat3)
    block7 = get_conv_block(dr3,nFilters*2,3)
    
    up4 = Conv2DTranspose(nFilters,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block7)
    b2_upsample = Conv2DTranspose(nFilters,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block2)
    cat4 = concatenate([block1, up4, b2_upsample])
    dr4 = Dropout(0.1)(cat4)
    block8 = get_conv_block(dr4,nFilters,3)

    conv10 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(block8)
    conv11 = Conv2D(4,(1,1), activation='softmax', padding = 'same',kernel_regularizer=l2(1e-4))(conv10)

    return (conv11, block7, block6, block5)

def get_model(input_shape,nFilters1,nFilters2):

    input_layer = Input(shape=input_shape)
    out1,out2,out3,out4 = get_UNET([input_layer],nFilters1,0)

    out1,out2,out3,out4 = get_UNET([out1,out2,out3,out4],nFilters2,1)

    model = Model(input_layer,out1)
    return model

In [0]:
PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/4 Class ER"

# Loading the data patient wise
X1 = np.load(PATH + '/ER IHC 230 Images.npy')
Y1 = np.load(PATH + '/ER IHC 230 Masks.npy')

X2 = np.load(PATH + '/ER IHC 232 Images.npy')
Y2 = np.load(PATH + '/ER IHC 232 Masks.npy')

X3 = np.load(PATH + '/ER IHC 242 Images.npy')
Y3 = np.load(PATH + '/ER IHC 242 Masks.npy')

X4 = np.load(PATH + '/ER IHC 263 Images.npy')
Y4 = np.load(PATH + '/ER IHC 263 Masks.npy')

X5 = np.load(PATH + '/ER IHC 221 Images.npy')
Y5 = np.load(PATH + '/ER IHC 221 Masks.npy')

X6 = np.load(PATH + '/ER IHC 229 Images.npy')
Y6 = np.load(PATH + '/ER IHC 229 Masks.npy')

X7 = np.load(PATH + '/ER IHC 239 Images.npy')
Y7 = np.load(PATH + '/ER IHC 239 Masks.npy')

X8 = np.load(PATH + '/ER IHC 246 Images.npy')
Y8 = np.load(PATH + '/ER IHC 246 Masks.npy')

X9 = np.load(PATH + '/ER IHC 252 Images.npy')
Y9 = np.load(PATH + '/ER IHC 252 Masks.npy')

In [0]:
X = [X1, X2, X3, X4, X5, X6, X7, X8, X9]
Y = [Y1, Y2, Y3, Y4, Y5, Y6, Y7, Y8, Y9]
patient_no = ['230','232','242','263','221','229','239','246','252']
size = [10,10,10,10,10,10,10,10,10]

In [0]:
fold_split = []

#test patient = 230, 232 
fold_split.append(([2,3,4,5,6,7,8],[0,1]))

#test_patient = 239, 242
fold_split.append(([0,1,3,4,5,7,8],[2,6]))

#test_patient = 252,263
fold_split.append(([0,1,2,4,5,6,7],[3,8]))

In [0]:
def convertToLabels(data):
  data[data==85]=1
  data[data==170]=2
  data[data==255]=3

def convertFromLabels(data):
  data[data==1]=85
  data[data==2]=170
  data[data==3]=255

In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

In [0]:
def tversky_loss_1(y_true, y_pred):
    alpha = 0.7
    beta  = 0.3
    
    ones = K.ones(K.shape(y_true))
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
  
    g0 = y_true
    g1 = ones-y_true
    
    num = K.sum(p0*g0, (0,1,2))
    den = num + alpha*K.sum(p0*g1,(0,1,2)) + beta*K.sum(p1*g0,(0,1,2))
    # 
    T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T

def tversky_loss_2(y_true, y_pred):
    alpha = 0.3
    beta  = 0.7
    
    ones = K.ones(K.shape(y_true))
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
    g0 = y_true
    g1 = ones-y_true
    
    num = K.sum(p0*g0, (0,1,2))
    den = num + alpha*K.sum(p0*g1,(0,1,2)) + beta*K.sum(p1*g0,(0,1,2))
    # 
    T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T

def focal_tversky_loss_1(y_true, y_pred, gamma=0.75):
    tversky_index = tversky_loss_1(y_true, y_pred)
    return K.pow((tversky_index), gamma)
  
def focal_tversky_loss_2(y_true, y_pred, gamma=0.75):
    tversky_index = tversky_loss_2(y_true, y_pred)
    return K.pow((tversky_index), gamma)

def combined_loss(y_true, y_pred):
  return (0.2*K.categorical_crossentropy(y_true, y_pred))+(0.8*focal_tversky_loss_1(y_true, y_pred)+(0.8*focal_tversky_loss_2(y_true, y_pred)))

In [0]:
batch_size = 8
def get_batch(batch_size, X_train, Y_train): 
    size_batch = batch_size
    last_index = len(X_train) - 1
    x_train = X_train
    y_train = Y_train 
    while True:
        batch_data = [[],[]]
        for i in range(0, size_batch):
            random_index = random.randint(0, last_index)
            batch_data[0].append(x_train[random_index])
            batch_data[1].append(y_train[random_index])

        yield (np.array(batch_data[0]), np.array(batch_data[1]))     

In [0]:
MODELS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Models/4 Class/ER/3Fold LadderNet"

fold=0

for train_index, test_index in fold_split:
  print("TRAIN:", train_index, "TEST:", test_index)
  
  TrainX =  np.concatenate(np.array([X[i] for i in train_index]))
  TrainY =  np.concatenate(np.array([Y[i] for i in train_index]))
  TestX  =  np.concatenate(np.array([X[i] for i in test_index]))
  TestY  =  np.concatenate(np.array([Y[i] for i in test_index]))

  Train = list(zip(TrainX,TrainY))
  random.shuffle(Train)
  TrainX,TrainY = zip(*Train)
  TrainX = np.asarray(list(TrainX))
  TrainY = np.asarray(list(TrainY))

  Test = list(zip(TestX,TestY))
  random.shuffle(Test)
  TestX,TestY = zip(*Test)
  TestX = np.asarray(list(TestX))
  TestY = np.asarray(list(TestY))

  TrainX = TrainX.astype('float32')/255
  TestX = TestX.astype('float32')/255
  convertToLabels(TrainY)
  TrainY = keras.utils.to_categorical(TrainY,num_classes=4,dtype='int16')
  convertToLabels(TestY)
  TestY = keras.utils.to_categorical(TestY,num_classes=4,dtype='int16')
  ValX = TrainX[(int)(0.8*TrainX.shape[0]):]
  ValY = TrainY[(int)(0.8*TrainY.shape[0]):]
  model = get_model((240,240,3),32,64)

  #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  mc = ModelCheckpoint('Checkpoint.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=True)

  model.compile(loss=combined_loss, optimizer= optimizer , metrics=[dice_coef,'accuracy']) 
  num_epoch = 100
  datagen = get_batch(batch_size, TrainX, TrainY)
  n_points = len(TrainX)
  print('-----------fold {}--------------'.format(fold))
  history = model.fit(datagen, validation_data = [ValX, ValY],
                  epochs=num_epoch,steps_per_epoch = math.ceil(n_points / batch_size), callbacks =[mc],  shuffle =True)
  model.save(MODELS_PATH + '/FoC_TVCombined_LadderNet_ER_v2_3'+ str(fold) +'.h5')
  fold = fold + 1

TRAIN: [2, 3, 4, 5, 6, 7, 8] TEST: [0, 1]
-----------fold 0--------------
Epoch 1/100
210/210 [==============================] - 180s 859ms/step - loss: 5.3113 - dice_coef: 0.3022 - accuracy: 0.8241 - val_loss: 5.3400 - val_dice_coef: 0.3266 - val_accuracy: 0.9233

Epoch 00001: val_loss improved from inf to 5.33995, saving model to Checkpoint.h5
Epoch 2/100
210/210 [==============================] - 168s 802ms/step - loss: 5.0585 - dice_coef: 0.3958 - accuracy: 0.9531 - val_loss: 4.9682 - val_dice_coef: 0.4404 - val_accuracy: 0.9408

Epoch 00002: val_loss improved from 5.33995 to 4.96822, saving model to Checkpoint.h5
Epoch 3/100
210/210 [==============================] - 169s 802ms/step - loss: 4.8091 - dice_coef: 0.4921 - accuracy: 0.9552 - val_loss: 4.6748 - val_dice_coef: 0.5467 - val_accuracy: 0.9431

Epoch 00003: val_loss improved from 4.96822 to 4.67478, saving model to Checkpoint.h5
Epoch 4/100
210/210 [==============================] - 168s 802ms/step - loss: 3.8786 - dice_coe

In [0]:
def stitchMaskPatches(pieces):
  k = 0
  reconstructed_img = np.ones([1440,1920])
  for r in range(6):
    row = r * 240
    for c in range(8):
      col = c * 240
      reconstructed_img[row:row+240,col:col+240] = pieces[k]
      k = k + 1
  return reconstructed_img


def stitchImagePatches(pieces):
  k = 0
  reconstructed_img = np.ones([1440,1920,3])
  for r in range(6):
    row = r * 240
    for c in range(8):
      col = c * 240
      reconstructed_img[row:row+240,col:col+240,:] = pieces[k]
      k = k + 1
  return reconstructed_img

In [0]:
def saveNumpyOutput(mask, Patient_array,Patient_length,title,folder):
  idx = 0
  for i in range(len(Patient_length)):
    temp = []
    for j in range(Patient_length[i]):
      final_output = mask[idx:idx+48]
      idx = idx + 48
      final_output = stitchMaskPatches(final_output)
      temp.append(final_output)
    final_output = np.asarray(temp)
    np.save(folder + title + Patient_array[i], final_output)

In [0]:
MODELS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Models/4 Class/ER/3Fold LadderNet/"
model_names = os.listdir(MODELS_PATH)
print(model_names)

['LadderNet_ER_v2_0.h5', 'LadderNet_ER_v2_1.h5', 'LadderNet_ER_v2_2.h5', 'FoC_TVCombined_LadderNet_ER_v2_30.h5']


In [0]:
SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/3Fold LadderNet/"

In [0]:
# Axis 0 = Folds
# Axis 1 = Class (Strong Intermediate,Weak,Background)
# Axis 2 = Evaluation Technique (Precision, Recall, Dice coefficient)
pixEvaluationTrain = np.empty((0,4,3))
pixEvaluationTest = np.empty((0,4,3))
score = np.zeros((1,4,3))

In [0]:
fold = 0

for train_index, test_index in fold_split:
  print("FOLD:",fold,"\tTRAIN:", train_index, "TEST:", test_index)
  # Splitting Train and Test data
  TrainX =  np.concatenate(np.array([X[i] for i in train_index]))
  TrainY =  np.concatenate(np.array([Y[i] for i in train_index]))
  TestX  =  np.concatenate(np.array([X[i] for i in test_index]))
  TestY  =  np.concatenate(np.array([Y[i] for i in test_index]))

  train_no = [patient_no[i] for i in train_index]
  train_size = [size[i] for i in train_index]

  test_no = [patient_no[i] for i in test_index]
  test_size = [size[i] for i in test_index]

  TrainX = TrainX.astype('float32')/255
  TestX = TestX.astype('float32')/255

  convertToLabels(TrainY)
  convertToLabels(TestY)

  # TrainY = keras.utils.to_categorical(TrainY,num_classes=4,dtype='int16')
  # TestY = keras.utils.to_categorical(TestY,num_classes=4,dtype='int16')
  
  # Loading corrosponding fold of model
  model = keras.models.load_model(MODELS_PATH + 'FoC_TVCombined_LadderNet_ER_v2_30.h5',custom_objects={ 'combined_loss': combined_loss, 'dice_coef': dice_coef })

  # Predicting results using model
  trainResult = model.predict(TrainX, batch_size=8)
  testResult = model.predict(TestX,batch_size=8)

  trainResult = np.argmax(trainResult,axis=-1)
  testResult = np.argmax(testResult,axis=-1)

  # trainOneHot = keras.utils.to_categorical(trainResult,num_classes=4,dtype='int16')
  # testOneHot = keras.utils.to_categorical(testResult,num_classes=4,dtype='int16')

  # Obtaining pixel-wise results: Precision recall and dice coefficient
  # For TRAIN
  y_true = np.reshape(TrainY,(-1))
  y_pred = np.reshape(trainResult,(-1))

  prec   = metrics.precision_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  recall = metrics.recall_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  dice = 2*prec*recall/(prec+recall)

  # Precision Recall Dice for each class
  for i in range(4):
    score[0][i][0] = prec[3-i]
    score[0][i][1] = recall[3-i]
    score[0][i][2] = dice[3-i]

  pixEvaluationTrain = np.append(pixEvaluationTrain,score,axis=0)
 
  # For TEST
  y_true = np.reshape(TestY,(-1))
  y_pred = np.reshape(testResult,(-1))

  prec   = metrics.precision_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  recall = metrics.recall_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  dice = 2*prec*recall/(prec+recall)

  # Precision Recall Dice for each class
  for i in range(4):
    score[0][i][0] = prec[3-i]
    score[0][i][1] = recall[3-i]
    score[0][i][2] = dice[3-i]

  pixEvaluationTest = np.append(pixEvaluationTest,score,axis=0)

  # Converting predicted labels to required output format
  # convertFromLabels(trainResult)
  # convertFromLabels(testResult)

  # # Saving numpy arrays
  # path = os.path.join(SAVE_PATH,"Fold " + str(fold))
  # os.mkdir(path)
  # os.mkdir(path + '/Train')
  # os.mkdir(path + '/Test')

  # saveNumpyOutput(trainResult, train_no, train_size,"/Train/",path)
  # saveNumpyOutput(testResult, test_no, test_size,"/Test/",path)

  fold +=1

FOLD: 0 	TRAIN: [2, 3, 4, 5, 6, 7, 8] TEST: [0, 1]


In [0]:
print(pixEvaluationTrain.shape)
print(pixEvaluationTest.shape)
print(pixEvaluationTest)

(1, 4, 3)
(1, 4, 3)
[[[0.88057365 0.38439159 0.53516902]
  [0.44256609 0.71980221 0.5481224 ]
  [0.46586629 0.1725451  0.25182178]
  [0.97250802 0.98358452 0.97801491]]]


In [0]:
np.save(SAVE_PATH + "Pixel-wise Metrics Train.npy", pixEvaluationTrain)
np.save(SAVE_PATH + "Pixel-wise Metrics Test.npy", pixEvaluationTest)

# **DISPLAY**

In [0]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


plt.figure(figsize=(20,20))
id = 48   # enter between 0- 50 since there are 5 patients with 10 images each

id = id * 48
final_input = TestX[id:id+48]
Mask_input = TestGT[id:id+48]
final_output = testResult[id:id+48]

final_input = stitchImagePatches(final_input)
Mask_input =  stitchMaskPatches(Mask_input)
final_output = stitchMaskPatches(final_output)

print(final_input.shape)
print(final_output.shape)
print(Mask_input.shape)

print(np.unique(final_output))
copy1  = np.copy(final_input)
copy2 = copy1.astype('float32')*255
copy2 = copy2.astype('uint8')
final_input = np.reshape(copy2,(1440, 1920,3))
final_input = cv2.cvtColor(final_input,cv2.COLOR_BGR2RGB)

plt.subplot(131).imshow(final_input)
plt.subplot(132).imshow(Mask_input,'gray')
plt.subplot(133).imshow(final_output,'gray')

In [0]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


plt.figure(figsize=(20,20))
id = 25  # enter between 0- 40 since there are 4 patients with 10 images each

id = id * 48
final_input = TrainX[id:id+48]
Mask_input = TrainGT[id:id+48]
final_output = trainResult[id:id+48]

final_input = stitchImagePatches(final_input)
Mask_input =  stitchMaskPatches(Mask_input)
final_output = stitchMaskPatches(final_output)

print(final_input.shape)
print(final_output.shape)
print(Mask_input.shape)

print(np.unique(final_output))
copy1  = np.copy(final_input)
copy2 = copy1.astype('float32')*255
copy2 = copy2.astype('uint8')
final_input = np.reshape(copy2,(1440, 1920,3))
final_input = cv2.cvtColor(final_input,cv2.COLOR_BGR2RGB)

plt.subplot(131).imshow(final_input)
plt.subplot(132).imshow(Mask_input,'gray')
plt.subplot(133).imshow(final_output,'gray')